In [1]:
# import dask.dataframe as dd
# import os

# # Указываем шаблон пути для файлов PSX
# psx_files_pattern = 'telecom100k/telecom100k/telecom100k/psx/*.csv'

# # Читаем файлы, указывая dtype для EndSession, чтобы гарантировать, что он читается как объект (строка)
# df_dask = dd.read_csv(psx_files_pattern, dtype={'EndSession': 'object'})

# # Если столбец с опечаткой 'Duartion' присутствует, переименовываем его в 'Duration'
# if 'Duartion' in df_dask.columns:
#     df_dask = df_dask.rename(columns={'Duartion': 'Duration'})

# # Вычисляем объединённый DataFrame, преобразуя Dask DataFrame в pandas DataFrame
# df_combined = df_dask.compute()

# # Сохраняем объединённый датасет в один CSV файл
# output_file = 'combined_psx.csv'
# df_combined.to_csv(output_file, index=False)

# print(f"Объединённый датасет сохранён в '{output_file}'")

In [2]:
# import pandas as pd

# df_combined = pd.read_csv('combined_psx.csv')
# df_combined

In [3]:
# # 2. Чтение файла с подписчиками (Subscriber)
# subscriber_file = 'telecom100k/telecom100k/telecom100k/subscribers.csv'
# df_subscriber = pd.read_csv(subscriber_file, dtype={'IdOnPSX': str, 'IdClient': str})

# # 3. Приведение типов для объединения
# df_combined['IdSubscriber'] = df_combined['IdSubscriber'].astype(str)
# df_subscriber['IdOnPSX'] = df_subscriber['IdOnPSX'].astype(str)

# # 4. Объединение по полям: PSXStats.IdSubscriber = Subscriber.IdOnPSX
# df_merged = pd.merge(df_combined, df_subscriber[['IdClient', 'IdOnPSX']], 
#                      left_on='IdSubscriber', right_on='IdOnPSX', how='left')

# # Убираем из итогового DataFrame лишний столбец (IdOnPSX) из subscriber
# df_merged.drop(columns=['IdOnPSX'], inplace=True)

# # 5. Сохраняем результат в новый CSV-файл
# output_file = 'combined_psx_with_client.csv'
# df_merged.to_csv(output_file, index=False)

# print(f"Новый датасет с добавленным IdClient сохранён в '{output_file}'")

In [4]:
# df_merged = pd.read_csv('combined_psx_with_client.csv')
# df_merged

In [5]:
# # Создаем папку для сохранения файлов, если её нет
# output_dir = 'clients'
# os.makedirs(output_dir, exist_ok=True)

# # Группируем данные по IdClient и сохраняем каждую группу в отдельный CSV-файл
# for client_id, group in df_merged.groupby('IdClient'):
#     # Сортировка по времени, если требуется (по StartSession)
#     group_sorted = group.sort_values('StartSession')
#     # Формируем имя файла для клиента
#     file_path = os.path.join(output_dir, f'client_{client_id}.csv')
#     group_sorted.to_csv(file_path, index=False)

# print("Датасет успешно разбит на группы по IdClient, файлы сохранены в папке 'clients'.")

In [ ]:
# import pandas as pd
# import glob
# import os

# # 1. Считываем исходный файл клиентов (client.parquet)
# # Здесь содержится информация о клиенте, включая идентификатор клиента и его тарифный план (IdPlan)
# clients_file = 'telecom100k/telecom100k/telecom100k/client.parquet'
# df_clients_orig = pd.read_parquet(clients_file)
# # Приводим идентификатор клиента к строковому типу и переименовываем для удобства объединения
# df_clients_orig['Id'] = df_clients_orig['Id'].astype(str)
# df_client_mapping = df_clients_orig[['Id', 'IdPlan']].copy()
# df_client_mapping.rename(columns={'Id': 'IdClient'}, inplace=True)
# df_client_mapping['IdPlan'] = df_client_mapping['IdPlan'].astype(str)

# # 2. Считываем данные тарифных планов (Plan) из JSON
# plan_file = 'telecom100k/telecom100k/telecom100k/plan.json'
# df_plan = pd.read_json(plan_file)
# df_plan['Id'] = df_plan['Id'].astype(str)  # тарифный план хранится в столбце 'Id' в df_plan

# # 3. Считываем данные параметров коммутаторов (PSXAttrs) из CSV
# psxattrs_file = 'telecom100k/telecom100k/telecom100k/psxattrs.csv'
# df_psxattrs = pd.read_csv(psxattrs_file, dtype={'Id': str})
# # Если нужно, можно оставить только интересующие столбцы:
# # df_psxattrs = df_psxattrs[['Id', 'PSX', 'TransmitUnits', 'Delimiter', 'DateFormat', 'TZ']]

# # 4. Находим все файлы клиентов в папке "clients"
# client_files = glob.glob(os.path.join('clients', 'client_*.csv'))

# # 5. Обрабатываем каждый файл
# for file in client_files:
#     # Считываем файл клиента. Файл содержит: IdSession, IdPSX, IdSubscriber, StartSession, EndSession, Duration, UpTx, DownTx, IdClient
#     df_client = pd.read_csv(file, dtype={'IdClient': str, 'IdPSX': str})
    
#     # 5.1. Объединяем с клиентским маппингом для добавления поля IdPlan
#     df_client = pd.merge(df_client, df_client_mapping, on='IdClient', how='left')
    
#     # 5.2. Объединяем с тарифными планами по полю IdPlan (df_client.IdPlan из маппинга = df_plan.Id)
#     df_client = pd.merge(df_client, df_plan, left_on='IdPlan', right_on='Id', how='left', suffixes=('', '_plan'))
#     # Если лишний столбец 'Id_plan' появился, его можно удалить (но здесь основное поле тарифного плана – 'IdPlan' уже добавлено)
    
#     # 5.3. Объединяем с параметрами коммутаторов (PSXAttrs) по полю IdPSX
#     df_client = pd.merge(df_client, df_psxattrs, left_on='IdPSX', right_on='Id', how='left', suffixes=('', '_psxattrs'))
#     # При необходимости можно удалить дублирующий столбец, например, 'Id_psxattrs'
#     if 'Id_psxattrs' in df_client.columns:
#         df_client.drop(columns=['Id_psxattrs'], inplace=True)
    
#     # 5.4. Сохраняем обновлённый DataFrame обратно в тот же файл
#     df_client.to_csv(file, index=False)

# print("Для каждого файла клиента добавлена информация о тарифном плане и PSXAttrs.")

In [7]:
import pandas as pd

def process_client_data(df):
    # Если в колонке TransmitUnits указано 'bytes', умножаем UpTx и DownTx на 8
    mask = df['TransmitUnits'] == 'bytes'
    df.loc[mask, 'UpTx'] = df.loc[mask, 'UpTx'] * 8
    df.loc[mask, 'DownTx'] = df.loc[mask, 'DownTx'] * 8

    # Функция для расчёта среднего количества пакетов за длительность
    def compute_avg_packets(row):
        if row['Duration'] <= 0:
            return 0
        return (row['UpTx'] + row['DownTx']) / row['Duration']
    
    # Применяем функцию и создаём столбец 'AvgPackets'
    df['AvgPackets'] = df.apply(compute_avg_packets, axis=1)

    # Функция для расчёта среднего исходящего трафика
    def compute_avg_up(row):
        if row['Duration'] <= 0:
            return 0
        return row['UpTx'] / row['Duration']
    
    # Функция для расчёта среднего входящего трафика
    def compute_avg_down(row):
        if row['Duration'] <= 0:
            return 0
        return row['DownTx'] / row['Duration']
    
    # Создаём столбцы 'AvgUp' и 'AvgDown'
    df['AvgUp'] = df.apply(compute_avg_up, axis=1)
    df['AvgDown'] = df.apply(compute_avg_down, axis=1)

    # Функция для извлечения второго значения из строки в колонке Attrs
    def get_second_attr(row):
        # Если Attrs или Delimiter отсутствуют, возвращаем None
        if pd.isna(row['Attrs']) or pd.isna(row['Delimiter']):
            return None
        parts = row['Attrs'].split(row['Delimiter'])
        # Проверяем, что есть как минимум два элемента
        if len(parts) >= 2:
            return parts[1]
        else:
            return None

    # Добавляем новый столбец с вторым значением
    df['EthernetPlan'] = df.apply(get_second_attr, axis=1)
    df = df.drop(columns=['Attrs', 'TransmitUnits', 'PSX', 'Delimiter', 'Description'])
    
    return df

    
# Пример использования:
# df = pd.read_csv('your_data.csv')
# df = process_client_data(df)

In [8]:
import pandas as pd
from datetime import datetime, timezone, timedelta

def convert_to_utc(date_str, date_format, tz_str):
    """
    Парсит дату по date_format и локализует по временной зоне из tz_str (например, 'GMT-6'),
    затем конвертирует время в UTC.
    """
    if pd.isna(date_str):
        return None
    dt = datetime.strptime(date_str, date_format)
    offset = int(tz_str.replace("GMT", ""))  # извлекаем смещение (например, -6)
    dt = dt.replace(tzinfo=timezone(timedelta(hours=offset)))
    return dt.astimezone(timezone.utc)

def process_session_columns(df):
    """
    1. Добавляет в DataFrame колонки StartSessionUTC и EndSessionUTC (переведённые в UTC).
    2. Создаёт колонку SessionDuration:
       - Если EndSessionUTC != 0, вычисляет разницу (в секундах) между EndSessionUTC и StartSessionUTC.
       - Если EndSessionUTC == 0 (EndSession NaN), берёт исходный Duration.
    3. Удаляет оригинальные колонки StartSession, EndSession, TZ, DateFormat.
    """

    # Преобразуем StartSession в UTC
    df['StartSessionUTC'] = df.apply(
        lambda row: convert_to_utc(row['StartSession'], row['DateFormat'], row['TZ']), axis=1
    )
    # Преобразуем EndSession в UTC или ставим 0
    df['EndSessionUTC'] = df.apply(
        lambda row: convert_to_utc(row['EndSession'], row['DateFormat'], row['TZ'])
        if pd.notna(row['EndSession']) else 0,
        axis=1
    )

    # Функция для вычисления фактической длительности сессии
    def compute_session_duration(row):
        # Если EndSessionUTC == 0, используем исходный Duration
        if row['EndSessionUTC'] == 0 or row['StartSessionUTC'] is None:
            return row['Duration']
        else:
            # Вычисляем разницу во времени (в секундах)
            return (row['EndSessionUTC'] - row['StartSessionUTC']).total_seconds()

    # Добавляем новую колонку с рассчитанной длительностью
    df['SessionDuration'] = df.apply(compute_session_duration, axis=1)

    # Удаляем ненужные колонки
    df = df.drop(columns=['StartSession', 'EndSession', 'TZ', 'Duration'])

    return df

In [9]:
def add_endpoint_column(df):
    """
    Функция принимает DataFrame с колонками:
      - StartSessionUTC: строковое представление даты/времени,
      - DateFormat: формат даты/времени (например, '%Y-%m-%d %H:%M:%S'),
      - duration: целое число секунд.
      
    Функция вычисляет новую колонку endpoint:
      endpoint = (StartSessionUTC + duration) в виде количества секунд с эпохи Unix.
    """
    
    def calc_endpoint(row):
        # Преобразуем StartSessionUTC в datetime с использованием указанного формата
        start_dt = pd.to_datetime(row['StartSessionUTC'], format=row['DateFormat'])
        # Прибавляем к дате duration в секундах
        end_dt = start_dt + pd.Timedelta(seconds=row['SessionDuration'])
        # Преобразуем итоговую дату в Unix timestamp (секунды с 1970-01-01)
        return int(end_dt.timestamp())
    
    # Применяем функцию построчно
    df['EndPoint'] = df.apply(calc_endpoint, axis=1)
    df = df.drop(columns=['EndSessionUTC', 'DateFormat'])
    return df

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def build_graph(df, x, y, bins=10, smoothing_window=6):
    """
    Функция строит график по заданным параметрам:
      - Сглаживает значения y с помощью скользящего среднего.
      - Отображает scatter-график сглаженных данных.
      - Строит непрерывную линию, показывающую медианные значения сглаженных y,
        рассчитанные по оконным (биновым) интервалам оси x.
    
    Параметры:
      df : pandas.DataFrame
          Датафрейм с данными.
      x : str
          Имя столбца для оси X (числовой или datetime).
      y : str
          Имя столбца для оси Y.
      bins : int, optional
          Количество бинов (окон) для расчёта медианы. По умолчанию 10.
      smoothing_window : int, optional
          Размер окна для сглаживания y (скользящее среднее). По умолчанию 5.
    """
    plt.figure(figsize=(10, 6))
    
    # Сглаживаем значения y
    df['y_smooth'] = df[y].rolling(window=smoothing_window, min_periods=1, center=True).mean()
    
    # Строим scatter-график сглаженных значений
    plt.scatter(df[x], df['y_smooth'], marker='o', label='Сглаженные данные')
    
    # Определяем границы бинов по оси x
    bin_edges = np.linspace(df[x].min(), df[x].max(), bins+1)
    
    # Группируем данные по созданным бинам
    df['bin'] = pd.cut(df[x], bins=bin_edges, include_lowest=True)
    
    # Вычисляем медианные значения сглаженных y для каждого бина
    median_df = df.groupby('bin', observed=False)['y_smooth'].median()
    
    # Вычисляем центры каждого бина
    bin_centers = [interval.mid for interval in median_df.index.categories]
    
    # Строим линию медианных значений
    plt.plot(bin_centers, median_df.values, color='red', marker='o', linestyle='-',
             label='Медиана по окнам (сглаженные)')
    
    # Вычисляем общую медиану сглаженных значений
    overall_median = df['y_smooth'].median()
    
    # Добавляем горизонтальную линию для общей медианы
    plt.axhline(y=overall_median, color='green', linestyle='--', label='Общая медиана (сглаженные)')
    
    plt.xlabel(x)
    plt.ylabel(y)
    plt.title(f"График сглаженных значений {y} по {x}")
    plt.legend()
    plt.grid(True)
    plt.show()

UpTx – это объём трафика, который передается от абонента (исходящий трафик).
DownTx – это объём трафика, который принимается абонентом (входящий трафик).

Для первичного анализа рекомендуется начать с UpTx.
Причина в том, что у обычных пользователей объём исходящего трафика часто значительно ниже, 
чем входящего, и любое резкое увеличение этого показателя может указывать на несанкционированную активность 
(например, утечку данных или взлом). Сравнивая ретроспективное значение UpTx с текущим, можно более чётко выявить аномалии в поведении абонента. 
После этого целесообразно проанализировать и DownTx для комплексной оценки ситуации.

In [ ]:
import os
import pandas as pd

def process_clients_files(folder_path):
    """
    Функция проходит по всем файлам в папке folder_path.
    Для каждого файла пытается загрузить данные в DataFrame и выполнить дальнейшую обработку.
    
    Параметры:
      folder_path : str
          Путь к папке, содержащей файлы клиентов.
    """
    count = 0
    # Получаем список всех файлов в папке
    for root, dirs, files in os.walk(folder_path):
        print(f"Найдено файлов: {len(files)}")
        for file in files:
            file_path = os.path.join(root, file)
            count += 1
            # Рисуем график только для каждого сотого файла
            if count % 100 == 0:
                try:
                    client = pd.read_csv(file_path)
                    
                    # Удаление лишних столбцов
                    client = client.drop(columns=['IdPSX', 'IdSubscriber', 'Name', 'Id', 'CreatedAt', 'CreatedAt', 'UpdatedAt', 'ClosedAt'])
                    
                    # Обработка данных: функции должны быть определены заранее
                    client = process_client_data(client)
                    client = process_session_columns(client)
                    client = add_endpoint_column(client)
                    
                    # Масштабирование столбцов 'EndPoint' и 'AvgPackets' в диапазоне от 0 до 1
                    for col in ['EndPoint', 'AvgPackets']:
                        min_val = client[col].min()
                        max_val = client[col].max()
                        # Если все значения равны, оставляем их без изменений, чтобы избежать деления на 0
                        if max_val != min_val:
                            client[col] = (client[col] - min_val) / (max_val - min_val)
                    
                    print(f'Обрабатываем файл {file_path}')
                    # Построение графика
                    build_graph(client, 'EndPoint', 'AvgPackets', bins=20, smoothing_window=5)
                except Exception as e:
                    print(f"Ошибка при обработке файла {file_path}: {e}")

# Пример вызова функции:
if __name__ == "__main__":
    folder = "clients"  # путь к папке с файлами
    process_clients_files(folder)

In [ ]:
client = pd.read_csv('clients/client_6bc922bf-d0c8-45bc-85a5-10bf446a00e1.csv')

client = client.drop(columns=['IdPSX', 'IdSubscriber', 'Name', 'Id', 'CreatedAt', 'CreatedAt', 'UpdatedAt', 'ClosedAt'])
client = process_client_data(client)
client = process_session_columns(client)
client = add_endpoint_column(client)

client.drop(columns=['IdPlan', 'IdClient', 'IdSession', 'StartSessionUTC', 'UpTx', 'DownTx', 'Enabled', 'EthernetPlan', 'SessionDuration']).sort_values('EndPoint')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

# Допустим, у вас уже есть DataFrame client
# где вы заранее удалили ненужные столбцы, например:
# client.drop(columns=['IdPlan','IdClient','IdSession','StartSessionUTC'], inplace=True, errors='ignore')

# 1. Формируем матрицу признаков
features = client[['AvgPackets', 'AvgUp', 'AvgDown', 'EndPoint']].copy()

# 2. Обучаем Isolation Forest
# Параметр contamination задаёт долю предполагаемых аномалий (например, 5%)
model = IsolationForest(contamination=0.05, random_state=42)
model.fit(features)

# 3. Получаем предсказания (-1 — аномалия, 1 — нормальная точка)
predictions = model.predict(features)

# 4. Добавляем колонку с флагом аномалии в исходный DataFrame
client['anomaly'] = (predictions == -1)

# 5. Визуализируем результат
# Для примера построим Scatter-график по осям: EndPoint (X) и AvgPackets (Y)
plt.figure(figsize=(10, 6))

# Нормальные точки
plt.scatter(
    client.loc[client['anomaly'] == False, 'EndPoint'],
    client.loc[client['anomaly'] == False, 'AvgPackets'],
    c='blue',
    label='Normal'
)

# Аномальные точки
plt.scatter(
    client.loc[client['anomaly'] == True, 'EndPoint'],
    client.loc[client['anomaly'] == True, 'AvgPackets'],
    c='red',
    label='Anomaly'
)

plt.xlabel('EndPoint')
plt.ylabel('AvgPackets')
plt.title('Isolation Forest - Anomaly Detection')
plt.legend()
plt.grid(True)
plt.show()

In [33]:
result = pd.read_csv('telecom100k/telecom100k/telecom100k/RESULT', sep=',')
client = pd.read_csv('clients/client_96969c99-55cc-4cb2-8227-063426eb6ee1.csv')
IdS = result['UID'].unique()  # Получаем уникальные значения Id


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

def process_clients_files(folder_path, files):
    """
    Функция проходит по всем файлам в папке folder_path.
    Для каждого файла загружает данные, выполняет обработку, обнаружение аномалий
    с использованием Isolation Forest и строит график.
    
    Параметры:
      folder_path : str
          Путь к папке, содержащей файлы клиентов.
      files : list
          Список идентификаторов/имен файлов (без расширения).
    """
    for file in files:
        file_path = os.path.join(folder_path, f'client_{file}.csv')
        try:
            # Загрузка данных
            client = pd.read_csv(file_path)
            
            # Удаление лишних столбцов
            client = client.drop(columns=['IdPSX', 'IdSubscriber', 'Name', 'Id', 
                                           'CreatedAt', 'UpdatedAt', 'ClosedAt'], errors='ignore')
            
            # Предварительная обработка данных (функции должны быть определены заранее)
            client = process_client_data(client)
            client = process_session_columns(client)
            client = add_endpoint_column(client)
            
            # Масштабирование столбцов 'EndPoint' и 'AvgPackets' в диапазоне [0, 1]
            for col in ['EndPoint', 'AvgPackets']:
                min_val = client[col].min()
                max_val = client[col].max()
                if max_val != min_val:
                    client[col] = (client[col] - min_val) / (max_val - min_val)
            
            print(f'Обрабатываем файл {file_path}')
            
            # === Обнаружение аномалий с Isolation Forest ===
            # 1. Формирование матрицы признаков
            features = client[['AvgPackets', 'AvgUp', 'AvgDown', 'EndPoint']].copy()
            
            # 2. Обучение модели Isolation Forest
            model = IsolationForest(contamination=0.05, random_state=42)
            model.fit(features)
            
            # 3. Получение предсказаний (-1 — аномалия, 1 — нормальная точка)
            predictions = model.predict(features)
            
            # 4. Добавление колонки с флагом аномалии
            client['anomaly'] = (predictions == -1)
            
            # === Визуализация результатов ===
            plt.figure(figsize=(10, 6))
            
            # Нормальные точки
            plt.scatter(
                client.loc[client['anomaly'] == False, 'EndPoint'],
                client.loc[client['anomaly'] == False, 'AvgPackets'],
                c='blue',
                label='Normal'
            )
            
            # Аномальные точки
            plt.scatter(
                client.loc[client['anomaly'] == True, 'EndPoint'],
                client.loc[client['anomaly'] == True, 'AvgPackets'],
                c='red',
                label='Anomaly'
            )
            
            plt.xlabel('EndPoint')
            plt.ylabel('AvgPackets')
            plt.title(f'Isolation Forest - Anomaly Detection for {file}')
            plt.legend()
            plt.grid(True)
            plt.show()
            
        except Exception as e:
            print(f"Ошибка при обработке файла {file_path}: {e}")

# Пример использования:
folder = "clients"  # путь к папке с файлами
process_clients_files(folder, IdS)